In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv
/kaggle/input/loan-datacube/appliaction_train_datacube.csv
/kaggle/input/loan-datacube/appliaction_test_datacube.csv


In [2]:
import sqlite3
import pandas as pd
import os

In [ ]:
csv_files = {
    "sample_submission": "/kaggle/input/home-credit-default-risk/sample_submission.csv",
    "bureau_balance": "/kaggle/input/home-credit-default-risk/bureau_balance.csv",
    "pos_cash_balance": "/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv",
    "application_train": "/kaggle/input/home-credit-default-risk/application_train.csv",
    "application_test": "/kaggle/input/home-credit-default-risk/application_test.csv",
    "previous_application": "/kaggle/input/home-credit-default-risk/previous_application.csv",
    "credit_card_balance": "/kaggle/input/home-credit-default-risk/credit_card_balance.csv",
    "installments_payments": "/kaggle/input/home-credit-default-risk/installments_payments.csv",
    "bureau": "/kaggle/input/home-credit-default-risk/bureau.csv"
}

In [ ]:
db_path = "home_credit.db"
conn = sqlite3.connect(db_path)

In [ ]:
for table_name, file_path in csv_files.items():
    print(f"Loading {table_name}...")
    
    df = pd.read_csv(file_path)
    df.to_sql(
        name=table_name,
        con=conn,
        if_exists="replace",  
        index=False
    )

In [ ]:
 tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)

print("Tables in database:")
print(tables)

In [ ]:
sql_1 = """
select * from application_train
limit 5;
"""

df_train = pd.read_sql(sql_1,conn)

In [ ]:
df_train

In [ ]:
sql_2 = """
create table customer_snajana_summary as
select SK_ID_CURR,
count(*) as lonas_bru,
sum(AMT_CREDIT_SUM) as total_sm_bre
from bureau
group by 1
"""

conn.execute(sql_2)
conn.commit()

In [ ]:
def r_sql(query,conn=conn):
    return pd.read_sql(query,conn)

In [ ]:
r_sql(sql_1)

In [ ]:
r_sql("select * from bureau_balance limit 5")

In [ ]:
r_sql("select distinct MONTHS_BALANCE from bureau_balance")

## Prod Queries

In [ ]:
sql_1_cr = """
--Agg bru_balance to bureau_level

create table bureau_balance_agg as 
select SK_ID_BUREAU,
min(MONTHS_BALANCE) as bb_months_balance_min,
max(MONTHS_BALANCE) as bb_months_balance_max,
count(*) as bb_months_balance_size,
sum(case when status = 'C' then 1 else 0 end) as bb_status_closed_count,
sum(case when status = '0' then 1 else 0 end) as bb_status_no_delay_count,
sum(case when status = '1' then 1 else 0 end) as bb_status_1_count,
sum(case when status in ('2','3','4','5') then 1 else 0 end) as bb_status_dpd_count,
sum(case when status in ('4','5') then 1 else 0 end) as bb_status_xtrm_count,
sum(case when cast(status as integer) > -7 then 1 else 0 end) as bb_status_gt_7_count
from bureau_balance
group by 1
"""
conn.execute(sql_1_cr)
conn.commit()

In [ ]:
sql_2_cr = """
create table bureau_withbalance as 
select b1.*,b2.* from bureau b1 left join bureau_balance_agg b2
on b1.sk_id_bureau=b2.sk_id_bureau 
"""

conn.execute(sql_2_cr)
conn.commit()

In [ ]:
sql_3_cr=""" 
-- agg to the SK_ID level
create table bureau_avg as
select sk_id_curr,
count(distinct sk_id_bureau) as bureau_count,
sum(case when credit_active="Active" then 1 else 0 end ) as Credit_Active,

min(days_credit) as bureau_dates_credit_min,
max(days_credit) as bureau_dates_credit_max,
avg(days_credit) as bureau_dates_credit_avg,

min(credit_day_overdue) as bureau_credit_day_overdue_credit_min,
max(credit_day_overdue) as bureau_credit_day_overdue_credit_max,
avg(credit_day_overdue) as bureau_credit_day_overdue_credit_avg,

min(days_credit_enddate) as bureau_days_credit_enddate_credit_min,
max(days_credit_enddate) as bureau_days_credit_enddate_credit_max,
avg(days_credit_enddate) as bureau_days_credit_enddate_credit_avg,

min(amt_credit_sum) as bureau_amt_credit_sum_credit_min,
max(amt_credit_sum) as bureau_amt_credit_sum_credit_max,
avg(amt_credit_sum) as bureau_amt_credit_sum_credit_avg,

sum(amt_annuity) as bureau_amt_annuity_credit_sum,
max(amt_annuity) as bureau_amt_annuity_credit_max,
avg(amt_annuity) as bureau_amt_annuity_credit_avg,

avg(bb_months_balance_size) as bureau_bb_months_balance_size_avg,
sum(bb_status_closed_count) as bureau_bb_status_closed_count_sum,
sum(bb_status_dpd_count) as bureau_bb_status_dpd_count_sum,
sum(bb_status_gt_7_count) as bureau_bb_status_gt_7_count_sum

from bureau_withbalance 
group by 1;
"""

conn.execute(sql_3_cr)
conn.commit()

In [ ]:
r_sql("select distinct name_contract_status from pos_cash_balance")

In [ ]:
sql_5_cr="""
-- agg_pos_cash to prev_application
create table pos_cash_balance_agg_1 as 
select 
sk_id_prev ,
count(*) as pos_month_size,
max(sk_dpd) as pos_sk_dpd_max,
avg(sk_dpd) as pos_sk_dpd_avg,
avg(CNT_INSTALMENT) as avg_cnt_installment,
avg(CNT_INSTALMENT_FUTURE) as avg_cnt_installment_future,
sum(case when name_contract_status="Active" then 1 else 0 end) as pos_active_count,
sum(case when name_contract_status="Completed" then 1 else 0 end) as pos_complete_count,
sum(case when name_contract_status="Amortized debt" then 1 else 0 end) as pos_debt_count
from pos_cash_balance
group by 1;
"""

conn.execute(sql_5_cr)
conn.commit()

In [ ]:
sql_7_cr = """
create table installments_enhanced
as
select 
*,
(AMT_PAYMENT - AMT_INSTALMENT) as payment_diff,
(days_entry_payment - days_instalment) as days_diff,
case when (days_entry_payment - days_instalment) > 0 then 1 else 0 end as late_payment
from installments_payments;
"""

conn.execute(sql_7_cr)
conn.commit()

In [ ]:
sql_8_cr = """
create table installments 
as
select 
sk_id_prev,
count(distinct NUM_INSTALMENT_VERSION) as inst_num_versions,

max(AMT_INSTALMENT) as inst_AMT_INSTALMENT_max,
min(AMT_INSTALMENT) as inst_AMT_INSTALMENT_min,
avg(AMT_INSTALMENT) as inst_AMT_INSTALMENT_avg,
sum(AMT_INSTALMENT) as inst_AMT_INSTALMENT_sum,

max(AMT_PAYMENT) as inst_AMT_PAYMENT_max,
min(AMT_PAYMENT) as inst_AMT_PAYMENT_min,
avg(AMT_PAYMENT) as inst_AMT_PAYMENT_avg,
sum(AMT_PAYMENT) as inst_AMT_PAYMENT_sum,

sum(late_payment) as inst_late_payment_sum,
avg(late_payment) as inst_late_payment_avg

from installments_enhanced
group by 1;
"""

conn.execute(sql_8_cr)
conn.commit()

In [ ]:
r_sql("select * from credit_card_balance limit 100")

In [ ]:
sql_10_cr = """
CREATE TABLE credit_card_agg_1 AS
SELECT
    SK_ID_PREV,
    MAX(SK_DPD) AS max_dpd,
    SUM(SK_DPD) AS sum_dpd,
    SUM(CASE WHEN SK_DPD > 0 and months_balance>-7 THEN 1 ELSE 0 END) AS count_dpd_months,
    AVG(CASE 
            WHEN AMT_INST_MIN_REGULARITY > 0 
            THEN AMT_PAYMENT_CURRENT * 1.0 / AMT_INST_MIN_REGULARITY 
        END) AS avg_payment_to_min,

    SUM(AMT_PAYMENT_CURRENT) AS total_payment,
    AVG(CASE 
            WHEN AMT_CREDIT_LIMIT_ACTUAL > 0 
            THEN AMT_BALANCE * 1.0 / AMT_CREDIT_LIMIT_ACTUAL 
        END) AS avg_util,

    MAX(CASE 
            WHEN AMT_CREDIT_LIMIT_ACTUAL > 0 
            THEN AMT_BALANCE * 1.0 / AMT_CREDIT_LIMIT_ACTUAL 
        END) AS max_util,

    -- LAST 6 MONTHS — UTIL
    AVG(CASE 
            WHEN months_balance>-7 and AMT_CREDIT_LIMIT_ACTUAL > 0
            THEN AMT_BALANCE * 1.0 / AMT_CREDIT_LIMIT_ACTUAL 
        END) AS l6m_avg_util,

    MAX(CASE 
            WHEN months_balance>-7  AND AMT_CREDIT_LIMIT_ACTUAL > 0
            THEN AMT_BALANCE * 1.0 / AMT_CREDIT_LIMIT_ACTUAL 
        END) AS l6m_max_util,

    SUM(CASE WHEN months_balance>-7  THEN AMT_DRAWINGS_ATM_CURRENT END) AS l6m_atm_draw,
    SUM(CASE WHEN months_balance>-7 THEN AMT_DRAWINGS_POS_CURRENT END) AS l6m_pos_draw,

   
    MAX(CNT_INSTALMENT_MATURE_CUM) AS total_installments,
    SUM(CASE WHEN NAME_CONTRACT_STATUS = 'Active' THEN 1 ELSE 0 END) AS months_active,

    SUM(CASE WHEN NAME_CONTRACT_STATUS = 'Completed' THEN 1 ELSE 0 END) AS months_completed

FROM credit_card_balance
GROUP BY SK_ID_PREV;
"""
conn.execute(sql_10_cr)
conn.commit()

In [ ]:
r_sql("select * from credit_card_agg_1 limit 5")

In [ ]:
sql_11_cr="""
create table pre_final_join as
select  pa.*,cc.*,i.*,pos.* from previous_application pa left join credit_card_agg_1 cc on pa.sk_id_prev=cc.sk_id_prev 

left join installments i on pa.sk_id_prev=i.sk_id_prev 

left join pos_cash_balance_agg_1 pos on pa.sk_id_prev=pos.sk_id_prev



"""
conn.execute(sql_11_cr)
conn.commit()

In [ ]:
pd.read_sql("select * from pre_final_join limit 5",conn).to_csv("sample.csv")

In [ ]:
sql_cr_12 = """
create table pre_join_2 
as
select 
a.*,
b.*
from application_train a left join bureau_avg b
on a.SK_ID_CURR = b.SK_ID_CURR
"""
conn.execute(sql_cr_12)
conn.commit()

In [ ]:
sql_cr_19 = """
create table pre_join_3
as
select 
a.*,
b.*
from application_test a left join bureau_avg b
on a.SK_ID_CURR = b.SK_ID_CURR
"""
conn.execute(sql_cr_19)
conn.commit()

r_sql("select * from pre_join_3 limit 5")

In [ ]:
r_sql("select * from pre_join_2 limit 5")

In [ ]:
r_sql("select distinct NAME_CLIENT_TYPE from previous_application")

In [ ]:
sql_12_cr='''
create table final_1 as 
select sk_id_curr,
sum(case when name_contract_type='Consumer loans' then 1 else 0 end) as count_consumer_loans,
sum(case when name_contract_type='Cash loans' then 1 else 0 end) as count_cash_loans,
sum(case when name_contract_type='Revolving loans' then 1 else 0 end) as count_Revolving_loans,
sum(case when name_contract_type='XNA' then 1 else 0 end) as count_XNA,

sum(case when name_contract_type='Consumer loans' then amt_annuity else 0 end) as consumer_annuity_sum,
sum(case when name_contract_type='Cash loans' then amt_annuity else 0 end) as cash_annuity_sum,
sum(case when name_contract_type='Revolving loans' then amt_annuity else 0 end) as revolving_annuity_sum,
sum(case when name_contract_type='XNA' then amt_annuity else 0 end) as XNA_annuity_sum,

sum(case when name_contract_type='Consumer loans' then AMT_APPLICATION else 0 end) as consumer_AMT_APPLICATION_sum,
sum(case when name_contract_type='Cash loans' then AMT_APPLICATION else 0 end) as cash_AMT_APPLICATION_sum,
sum(case when name_contract_type='Revolving loans' then AMT_APPLICATION else 0 end) as revolving_AMT_APPLICATION_sum,
sum(case when name_contract_type='XNA' then AMT_APPLICATION else 0 end) as XNA_AMT_APPLICATION_sum,

sum(case when name_contract_type='Consumer loans' then AMT_CREDIT else 0 end) as consumer_AMT_CREDIT_sum,
sum(case when name_contract_type='Cash loans' then AMT_CREDIT else 0 end) as cash_AMT_CREDIT_sum,
sum(case when name_contract_type='Revolving loans' then AMT_CREDIT else 0 end) as revolving_AMT_CREDIT_sum,
sum(case when name_contract_type='XNA' then AMT_CREDIT else 0 end) as XNA_AMT_CREDIT_sum,

sum(case when name_contract_type='Consumer loans' then AMT_GOODS_PRICE else 0 end) as consumer_AMT_GOODS_PRICE_sum,
sum(case when name_contract_type='Cash loans' then AMT_GOODS_PRICE else 0 end) as cash_AMT_GOODS_PRICE_sum,
sum(case when name_contract_type='Revolving loans' then AMT_GOODS_PRICE else 0 end) as revolving_AMT_GOODS_PRICE_sum,
sum(case when name_contract_type='XNA' then AMT_GOODS_PRICE else 0 end) as XNA_AMT_GOODS_PRICE_sum,

sum(case when NAME_CLIENT_TYPE='Repeater' then 1 else 0 end) as count_Repeater,
sum(case when NAME_CLIENT_TYPE='New' then 1 else 0 end) as count_New,
sum(case when NAME_CLIENT_TYPE='Refreshed' then 1 else 0 end) as count_Refreshed,
sum(case when NAME_CLIENT_TYPE='XNA' then 1 else 0 end) as count_XNA_ct,

avg(avg_payment_to_min) as avg_payment_to_min,
sum(total_payment) as total_payment,
avg(avg_util) as avg_util,
max(max_util) as max_util,
avg(l6m_avg_util) as l6m_max_util,
max(l6m_max_util) as l6m_max_util,
sum(l6m_atm_draw) as l6m_atm_draw,
sum(l6m_pos_draw) as l6m_pos_draw,
sum(total_installments) as total_installments,
sum(months_active) as credit_loan_months_active,
sum(months_completed) as credit_loan_months_completed,

sum(inst_num_versions) as inst_num_versions,
max(inst_AMT_INSTALMENT_max) as inst_AMT_INSTALMENT_max,
min(inst_AMT_INSTALMENT_min) as inst_AMT_INSTALMENT_min,
avg(inst_AMT_INSTALMENT_min) as inst_AMT_INSTALMENT_min,
sum(inst_AMT_INSTALMENT_min) as inst_AMT_INSTALMENT_min,

max(inst_AMT_PAYMENT_max) as inst_AMT_PAYMENT_max,
min(inst_AMT_PAYMENT_min) as inst_AMT_PAYMENT_min,
avg(inst_AMT_PAYMENT_avg) as inst_AMT_PAYMENT_avg,
sum(inst_AMT_PAYMENT_sum) as inst_AMT_PAYMENT_sum,
sum(inst_late_payment_sum) as inst_late_payment_sum,
avg(inst_late_payment_avg) as inst_late_payment_avg,

sum(pos_month_size) as pos_month_size,
max(pos_sk_dpd_max) as pos_sk_dpd_max,
avg(pos_sk_dpd_avg) as pos_sk_dpd_avg,
avg(avg_cnt_installment) as avg_cnt_installment,
avg(avg_cnt_installment_future) as avg_cnt_installment_future,
sum(pos_active_count) as pos_active_count,
sum(pos_complete_count) as pos_complete_count,
sum(pos_debt_count) as pos_debt_count

from pre_final_join
group by 1

'''

conn.execute(sql_12_cr)
conn.commit()

In [ ]:
r_sql("select * from final_1 limit 5")

In [ ]:
sql_final_cr = """
create table sql_final_table
as
select
a.*,
b.*
from pre_join_2 a left join final_1 b
on a.sk_id_curr = b.sk_id_curr
"""

conn.execute(sql_final_cr)
conn.commit()

In [ ]:
sql_final_cr_test = """
create table sql_final_table_test
as
select
a.*,
b.*
from pre_join_3 a left join final_1 b
on a.sk_id_curr = b.sk_id_curr
"""

conn.execute(sql_final_cr_test)
conn.commit()

r_sql("select * from sql_final_table_test limit 5")

In [ ]:
r_sql("select * from  sql_final_table limit 10")

In [ ]:
pd.read_sql("select * from sql_final_table",conn).to_csv("appliaction_train_datacube.csv")

In [ ]:
pd.read_sql("select * from sql_final_table_test",conn).to_csv("appliaction_test_datacube.csv")

In [3]:
train_data = pd.read_sql("select * from sql_final_table",conn)
test_data = pd.read_sql("select * from sql_final_table_test",conn)

NameError: name 'conn' is not defined

In [84]:
train_data = pd.read_csv("/kaggle/input/loan-datacube/appliaction_train_datacube.csv")
test_data = pd.read_csv("/kaggle/input/loan-datacube/appliaction_test_datacube.csv")

In [85]:
print(train_data.shape)
print(test_data.shape)

(307511, 200)
(48744, 199)


In [86]:
train_data['TARGET'].value_counts()

TARGET
0    282686
1     24825
Name: count, dtype: int64

In [87]:
train_data.dtypes.value_counts()

float64    142
int64       42
object      16
Name: count, dtype: int64

In [88]:
len(train_data.columns)

200

In [89]:
len(set(train_data.columns))

200

In [90]:
train_data.drop(columns=['Unnamed: 0'],inplace=True)

In [91]:
train_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,inst_late_payment_sum,inst_late_payment_avg,pos_month_size,pos_sk_dpd_max,pos_sk_dpd_avg,avg_cnt_installment,avg_cnt_installment_future,pos_active_count,pos_complete_count,pos_debt_count
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.000000,19.0,0.0,0.0,24.000000,15.000000,19.0,0.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.000000,28.0,0.0,0.0,9.791667,5.666667,26.0,2.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.000000,4.0,0.0,0.0,3.750000,2.250000,3.0,1.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.000000,21.0,0.0,0.0,12.888889,10.214286,18.0,2.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,16.0,0.287912,66.0,0.0,0.0,15.066667,8.966667,62.0,3.0,0.0


In [92]:
test_data.head()

,Unnamed: 0,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,inst_late_payment_sum,inst_late_payment_avg,pos_month_size,pos_sk_dpd_max,pos_sk_dpd_avg,avg_cnt_installment,avg_cnt_installment_future,pos_active_count,pos_complete_count,pos_debt_count
0,0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,...,0.0,0.000000,5.0,0.0,0.000000,4.000000,2.000000,4.0,1.0,0.0
1,1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,...,1.0,0.111111,11.0,0.0,0.000000,11.700000,7.200000,9.0,1.0,0.0
2,2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,...,10.0,0.194444,36.0,18.0,1.619048,16.648148,11.451178,32.0,3.0,0.0
3,3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,...,12.0,0.047619,31.0,0.0,0.000000,15.928571,8.312500,29.0,2.0,0.0
4,4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,...,0.0,0.000000,13.0,0.0,0.000000,12.000000,5.846154,12.0,1.0,0.0


In [93]:
test_data.drop(columns=['Unnamed: 0'],inplace=True)

In [94]:
test_data.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,inst_late_payment_sum,inst_late_payment_avg,pos_month_size,pos_sk_dpd_max,pos_sk_dpd_avg,avg_cnt_installment,avg_cnt_installment_future,pos_active_count,pos_complete_count,pos_debt_count
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0.0,0.000000,5.0,0.0,0.000000,4.000000,2.000000,4.0,1.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,1.0,0.111111,11.0,0.0,0.000000,11.700000,7.200000,9.0,1.0,0.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,10.0,0.194444,36.0,18.0,1.619048,16.648148,11.451178,32.0,3.0,0.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,12.0,0.047619,31.0,0.0,0.000000,15.928571,8.312500,29.0,2.0,0.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0.0,0.000000,13.0,0.0,0.000000,12.000000,5.846154,12.0,1.0,0.0


In [95]:
train_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,inst_late_payment_sum,inst_late_payment_avg,pos_month_size,pos_sk_dpd_max,pos_sk_dpd_avg,avg_cnt_installment,avg_cnt_installment_future,pos_active_count,pos_complete_count,pos_debt_count
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.000000,19.0,0.0,0.0,24.000000,15.000000,19.0,0.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.000000,28.0,0.0,0.0,9.791667,5.666667,26.0,2.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.000000,4.0,0.0,0.0,3.750000,2.250000,3.0,1.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.000000,21.0,0.0,0.0,12.888889,10.214286,18.0,2.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,16.0,0.287912,66.0,0.0,0.0,15.066667,8.966667,62.0,3.0,0.0


In [96]:
train_data.columns.tolist()

['SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_A

In [97]:
train_data.drop(columns=['SK_ID_CURR:1','SK_ID_CURR:2'],inplace=True)

In [98]:
test_data.drop(columns=['SK_ID_CURR:1','SK_ID_CURR:2'],inplace=True)

In [99]:
train_data.shape

(307511, 197)

In [100]:
TARGET = 'TARGET'
ID_COL = 'SK_ID_CURR'

exc_cols = [TARGET,ID_COL]
feature_cols = [c for c in train_data.columns if c not in exc_cols]

In [101]:
numeric_cols = []
cat_cols = []

for col in feature_cols:
    if train_data[col].dtype in ['int64','float64']:
        n_unique = train_data[col].nunique()
        if n_unique <= 10 and train_data[col].dtype == 'int64':
            cat_cols.append(col)
        else:
            numeric_cols.append(col)
    else:
        cat_cols.append(col)

print(len(numeric_cols))
print(len(cat_cols))

145
50


In [102]:
missing_stats = pd.DataFrame({
    'col': train_data.columns,
    'msc': train_data.isnull().sum(),
    'msc_pct': (train_data.isnull().sum() / len(train_data) * 100).round(2),
    'dtype': train_data.dtypes
})
missing_stats

,col,msc,msc_pct,dtype
SK_ID_CURR,SK_ID_CURR,0,0.00,int64
TARGET,TARGET,0,0.00,int64
NAME_CONTRACT_TYPE,NAME_CONTRACT_TYPE,0,0.00,object
CODE_GENDER,CODE_GENDER,0,0.00,object
FLAG_OWN_CAR,FLAG_OWN_CAR,0,0.00,object
...,...,...,...,...
avg_cnt_installment,avg_cnt_installment,20576,6.69,float64
avg_cnt_installment_future,avg_cnt_installment_future,20576,6.69,float64
pos_active_count,pos_active_count,20544,6.68,float64
pos_complete_count,pos_complete_count,20544,6.68,float64


In [103]:
missing_stats = missing_stats[missing_stats['msc'] > 0].sort_values('msc_pct',ascending=False)
missing_stats.to_csv("missing_data.csv",index=False)

In [104]:
high_treshold = 45
hcs = missing_stats[missing_stats['msc'] > high_treshold]['col'].tolist()

train_clean = train_data.drop(columns = hcs)
test_clean = test_data.drop(columns = hcs)

In [105]:
hcs

['avg_payment_to_min',
 'total_payment',
 'l6m_atm_draw',
 'l6m_pos_draw',
 'l6m_max_util:1',
 'l6m_max_util',
 'avg_util',
 'max_util',
 'credit_loan_months_completed',
 'total_installments',
 'credit_loan_months_active',
 'bureau_amt_annuity_credit_sum',
 'bureau_amt_annuity_credit_avg',
 'bureau_amt_annuity_credit_max',
 'bureau_bb_status_dpd_count_sum',
 'bureau_bb_status_gt_7_count_sum',
 'bureau_bb_months_balance_size_avg',
 'bureau_bb_status_closed_count_sum',
 'COMMONAREA_MODE',
 'COMMONAREA_AVG',
 'COMMONAREA_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAPARTMENTS_AVG',
 'FONDKAPREMONT_MODE',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAPARTMENTS_MODE',
 'FLOORSMIN_MODE',
 'FLOORSMIN_MEDI',
 'FLOORSMIN_AVG',
 'YEARS_BUILD_AVG',
 'YEARS_BUILD_MEDI',
 'YEARS_BUILD_MODE',
 'OWN_CAR_AGE',
 'LANDAREA_MEDI',
 'LANDAREA_AVG',
 'LANDAREA_MODE',
 'BASEMENTAREA_MODE',
 'BASEMENTAREA_MEDI',
 'BASEMENTAREA_AVG',
 'EXT_SOURCE_1',
 'NONLIVINGARE

In [106]:
missing_indiactor_cols = []
for col in numeric_cols:
    missing_pct = train_data[col].isnull().sum() / len(train_clean) * 100
    if missing_pct < 40:
        indicator_col = f'{col}_was_missing'
        train_clean[indicator_col] = train_data[col].isnull().astype(int)
        test_clean[indicator_col] = train_data[col].isnull().astype(int)

        missing_indiactor_cols.append(indicator_col)

In [107]:
train_clean.shape

(307511, 140)

In [108]:
feature_cols_1 = [c for c in train_clean.columns if c not in exc_cols]
numeric_cols_1 = []
cat_cols_1 = []

for col in feature_cols_1:
    if train_clean[col].dtype in ['int64','float64']:
        n_unique = train_clean[col].nunique()
        if n_unique <= 10 and train_clean[col].dtype == 'int64':
            cat_cols_1.append(col)
        else:
            numeric_cols_1.append(col)
    else:
        cat_cols_1.append(col)

print(len(numeric_cols_1))
print(len(cat_cols_1))

12
126


In [109]:
import numpy as np
from sklearn.impute import KNNImputer

In [110]:
low_missing_num = []
high_missing_num = []

for col in numeric_cols_1:
    if train_clean[col].isnull().any():
        missing_pct = train_clean[col].isnull().sum() / len(train_clean) * 100
        if missing_pct < 10:
            low_missing_num.append(col)
        else:
            high_missing_num.append(col)

if low_missing_num:
    knn_imputer = KNNImputer(n_neighbors=5,weights='distance')
    train_clean[low_missing_num] = knn_imputer.fit_transform(train_clean[low_missing_num])
    test_clean[low_missing_num] = knn_imputer.fit_transform(test_clean[low_missing_num])


if high_missing_num:
    for col in high_missing_num:
        median_value = train_clean[col].median()
        train_clean[col].fillna(median_value,inplace=True)
        test_clean[col].fillna(median_value,inplace=True)
    

In [111]:
for col in cat_cols_1:
    if train_clean[col].isnull().any():
        mode_value = train_clean[col].mode()[0]
        train_clean[col].fillna(mode_value,inplace=True)
        test_clean[col].fillna(mode_value,inplace=True)

In [112]:
train_clean.isnull().sum().sum()

np.int64(0)

In [113]:
test_clean.isnull().sum().sum()

np.int64(0)

In [114]:
outlier_summary = []
outiler_bounds = {}

for col in numeric_cols_1:
    Q1 = train_clean[col].quantile(0.25)
    Q3 = train_clean[col].quantile(0.75)
    IQR = Q3 - Q1

    lb = Q1 - 3 * IQR
    ub = Q3 + 3 * IQR

    outliers = ((train_clean[col] < lb) | (train_clean[col] > ub)).sum()

    if outliers > 0:
        o_pct = (outliers / len(train_clean)) * 100

        lower_cap = train_clean[col].quantile(0.01)
        upper_cap = train_clean[col].quantile(0.99)

        outiler_bounds[col] = (lower_cap,upper_cap)

        outlier_summary.append(
            {
                'col':col,
                'outliers': outliers,
                'out_pct': round(o_pct,2),
                'l_cap': round(lower_cap,2),
                'u_cap': round(upper_cap,2)
            }
        )

out_data_train_clean = pd.DataFrame(outlier_summary)

In [115]:
out_data_train_clean

,col,outliers,out_pct,l_cap,u_cap
0,CNT_CHILDREN,126,0.04,0.0,3.0
1,AMT_INCOME_TOTAL,3014,0.98,45000.0,472500.0
2,AMT_CREDIT,444,0.14,76410.0,1854000.0
3,AMT_ANNUITY,1024,0.33,6183.0,70006.5
4,DAYS_EMPLOYED,59624,19.39,-10894.9,365243.0
5,DAYS_REGISTRATION,1,0.00,-13879.0,-50.0
6,CNT_FAM_MEMBERS,121,0.04,1.0,5.0


In [116]:
for col, (lower_cap,upper_cap) in outiler_bounds.items():
    train_clean[col] = train_clean[col].clip(lower=lower_cap,upper=upper_cap)

In [117]:
train_clean

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,inst_late_payment_sum_was_missing,inst_late_payment_avg_was_missing,pos_month_size_was_missing,pos_sk_dpd_max_was_missing,pos_sk_dpd_avg_was_missing,avg_cnt_installment_was_missing,avg_cnt_installment_future_was_missing,pos_active_count_was_missing,pos_complete_count_was_missing,pos_debt_count_was_missing
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0,0,0,0,0,0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0,0,0,0,0,0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,0,0,0,0,0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,0,0,0,0,0,0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,0,0,0,0,0,0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,0,0,0,0,0,0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0,0,0,0,0,0


- Feature Eng
- Cateogrical Handling
- Feature Scaling
- Feature Importance (Univariate Analysis)
- Multi Collinearlity (VIF)
- Feature Stability Test
- Data Leakage

In [118]:
train_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,inst_late_payment_sum,inst_late_payment_avg,pos_month_size,pos_sk_dpd_max,pos_sk_dpd_avg,avg_cnt_installment,avg_cnt_installment_future,pos_active_count,pos_complete_count,pos_debt_count
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.000000,19.0,0.0,0.0,24.000000,15.000000,19.0,0.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.000000,28.0,0.0,0.0,9.791667,5.666667,26.0,2.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.000000,4.0,0.0,0.0,3.750000,2.250000,3.0,1.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.000000,21.0,0.0,0.0,12.888889,10.214286,18.0,2.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,16.0,0.287912,66.0,0.0,0.0,15.066667,8.966667,62.0,3.0,0.0


In [119]:
test_data.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,inst_late_payment_sum,inst_late_payment_avg,pos_month_size,pos_sk_dpd_max,pos_sk_dpd_avg,avg_cnt_installment,avg_cnt_installment_future,pos_active_count,pos_complete_count,pos_debt_count
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0.0,0.000000,5.0,0.0,0.000000,4.000000,2.000000,4.0,1.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,1.0,0.111111,11.0,0.0,0.000000,11.700000,7.200000,9.0,1.0,0.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,10.0,0.194444,36.0,18.0,1.619048,16.648148,11.451178,32.0,3.0,0.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,12.0,0.047619,31.0,0.0,0.000000,15.928571,8.312500,29.0,2.0,0.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0.0,0.000000,13.0,0.0,0.000000,12.000000,5.846154,12.0,1.0,0.0


In [120]:
print("Engineering financial ratio features")

# Credit to Annuity Ratio
if 'AMT_CREDIT' in train_clean.columns and 'AMT_ANNUITY' in train_clean.columns:
    train_clean['credit_annuity_ratio'] = (
        train_clean['AMT_CREDIT'] / (train_clean['AMT_ANNUITY'] + 1)
    )
    test_clean['credit_annuity_ratio'] = (
        test_clean['AMT_CREDIT'] / (test_clean['AMT_ANNUITY'] + 1)
    )

# Credit to Income Ratio
if 'AMT_INCOME_TOTAL' in train_clean.columns and 'AMT_CREDIT' in train_clean.columns:
    train_clean['credit_income_ratio'] = (
        train_clean['AMT_CREDIT'] / (train_clean['AMT_INCOME_TOTAL'] + 1)
    )
    test_clean['credit_income_ratio'] = (
        test_clean['AMT_CREDIT'] / (test_clean['AMT_INCOME_TOTAL'] + 1)
    )

# Annuity to Income Ratio (Debt Burden)
if 'AMT_INCOME_TOTAL' in train_clean.columns and 'AMT_ANNUITY' in train_clean.columns:
    train_clean['annuity_income_ratio'] = (
        train_clean['AMT_ANNUITY'] / (train_clean['AMT_INCOME_TOTAL'] + 1)
    )
    test_clean['annuity_income_ratio'] = (
        test_clean['AMT_ANNUITY'] / (test_clean['AMT_INCOME_TOTAL'] + 1)
    )

# Goods Price to Credit Ratio
if 'AMT_GOODS_PRICE' in train_clean.columns and 'AMT_CREDIT' in train_clean.columns:
    train_clean['goods_credit_ratio'] = (
        train_clean['AMT_GOODS_PRICE'] / (train_clean['AMT_CREDIT'] + 1)
    )
    test_clean['goods_credit_ratio'] = (
        test_clean['AMT_GOODS_PRICE'] / (test_clean['AMT_CREDIT'] + 1)
    )

print("Financial ratio features created")

Engineering financial ratio features
Financial ratio features created


In [121]:
# Age in years
if 'DAYS_BIRTH' in train_clean.columns:
    train_clean['age_years'] = (-train_clean['DAYS_BIRTH'] / 365).astype(int)
    test_clean['age_years'] = (-test_clean['DAYS_BIRTH'] / 365).astype(int)

    # Age groups
    train_clean['age_group_young'] = (train_clean['age_years'] < 35).astype(int)
    train_clean['age_group_middle'] = (
        (train_clean['age_years'] >= 35) & (train_clean['age_years'] < 55)
    ).astype(int)
    train_clean['age_group_senior'] = (train_clean['age_years'] >= 55).astype(int)

    test_clean['age_group_young'] = (test_clean['age_years'] < 35).astype(int)
    test_clean['age_group_middle'] = (
        (test_clean['age_years'] >= 35) & (test_clean['age_years'] < 55)
    ).astype(int)
    test_clean['age_group_senior'] = (test_clean['age_years'] >= 55).astype(int)

# Employment duration in years
if 'DAYS_EMPLOYED' in train_clean.columns:
    train_clean['employment_years'] = -train_clean['DAYS_EMPLOYED'] / 365
    test_clean['employment_years'] = -test_clean['DAYS_EMPLOYED'] / 365

    # Handle anomalies (365243 days indicates unemployed or pension)
    train_clean.loc[train_clean['employment_years'] > 100, 'employment_years'] = np.nan
    test_clean.loc[test_clean['employment_years'] > 100, 'employment_years'] = np.nan

    # Impute missing values using training median
    emp_median = train_clean['employment_years'].median()
    train_clean['employment_years'].fillna(emp_median, inplace=True)
    test_clean['employment_years'].fillna(emp_median, inplace=True)

# Years since registration
if 'DAYS_REGISTRATION' in train_clean.columns:
    train_clean['registration_years'] = -train_clean['DAYS_REGISTRATION'] / 365
    test_clean['registration_years'] = -test_clean['DAYS_REGISTRATION'] / 365

print("Age and employment features created")

Age and employment features created


/tmp/ipykernel_55/337003207.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_clean['age_group_middle'] = (
/tmp/ipykernel_55/337003207.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_clean['age_group_senior'] = (train_clean['age_years'] >= 55).astype(int)
/tmp/ipykernel_55/337003207.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get

In [122]:
ext_source_cols = [c for c in train_clean.columns if 'EXT_SOURCE' in c]

if len(ext_source_cols) >= 2:
    train_clean['ext_source_mean'] = train_clean[ext_source_cols].mean(axis=1)
    train_clean['ext_source_max'] = train_clean[ext_source_cols].max(axis=1)
    train_clean['ext_source_min'] = train_clean[ext_source_cols].min(axis=1)
    train_clean['ext_source_std'] = train_clean[ext_source_cols].std(axis=1)

    test_clean['ext_source_mean'] = test_clean[ext_source_cols].mean(axis=1)
    test_clean['ext_source_max'] = test_clean[ext_source_cols].max(axis=1)
    test_clean['ext_source_min'] = test_clean[ext_source_cols].min(axis=1)
    test_clean['ext_source_std'] = test_clean[ext_source_cols].std(axis=1)

# Document submission count
doc_cols = [c for c in train_clean.columns if 'FLAG_DOCUMENT' in c]
if doc_cols:
    train_clean['total_documents_provided'] = train_clean[doc_cols].sum(axis=1)
    test_clean['total_documents_provided'] = test_clean[doc_cols].sum(axis=1)

# Bureau-related ratios
if 'bureau_count' in train_clean.columns and 'Credit_Active' in train_clean.columns:
    train_clean['active_credit_ratio'] = (
        train_clean['Credit_Active'] / (train_clean['bureau_count'] + 1)
    )
    test_clean['active_credit_ratio'] = (
        test_clean['Credit_Active'] / (test_clean['bureau_count'] + 1)
    )

print("Additional features created")
print(f"Current dataset shape: {train_clean.shape}")

/tmp/ipykernel_55/677573230.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_clean['ext_source_mean'] = train_clean[ext_source_cols].mean(axis=1)
/tmp/ipykernel_55/677573230.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_clean['ext_source_max'] = train_clean[ext_source_cols].max(axis=1)
/tmp/ipykernel_55/677573230.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

Additional features created
Current dataset shape: (307511, 154)


/tmp/ipykernel_55/677573230.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_clean['total_documents_provided'] = train_clean[doc_cols].sum(axis=1)
/tmp/ipykernel_55/677573230.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_clean['total_documents_provided'] = test_clean[doc_cols].sum(axis=1)


In [123]:
train_clean.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,age_group_young,age_group_middle,age_group_senior,employment_years,registration_years,ext_source_mean,ext_source_max,ext_source_min,ext_source_std,total_documents_provided
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,1,0,0,1.745205,9.994521,0.0,0,0,0.000000,1
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,1,0,3.254795,3.249315,0.5,1,0,0.707107,1
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,1,0,0.616438,11.671233,0.0,0,0,0.000000,0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,1,0,8.326027,26.939726,0.5,1,0,0.707107,1
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,1,0,8.323288,11.810959,0.5,1,0,0.707107,1


In [124]:
categorical_cols_updated = [c for c in train_clean.columns if c not in [TARGET,ID_COL] and
                            (train_clean[c].dtype == 'object')]
categorical_cols_updated

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE']

In [125]:
categorical_cols_updated_2 = [c for c in train_clean.columns if c not in [TARGET,ID_COL] and
                            (train_clean[c].dtype == 'object')]
categorical_cols_updated_2

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE']

In [126]:
for col in categorical_cols_updated:
    n_un = train_clean[col].nunique()
    print(col)

NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE


In [127]:
col = 'ORGANIZATION_TYPE'
target_mean = train_clean.groupby(col)[TARGET].mean()
global_mean = train_clean[TARGET].mean()

smoothing = 10
counts = train_clean[col].value_counts()

smoothed_encoding = {}
for cat in target_mean.index:
    n = counts.get(cat,0)
    smoothed_encoding[cat] = (target_mean[cat] * n + global_mean * smoothing) / (n + smoothing)
    encoded_col_name = f'{col}_encoded'

train_clean[encoded_col_name] = train_clean[col].map(smoothed_encoding).fillna(global_mean)
test_clean[encoded_col_name] = train_clean[col].map(smoothed_encoding).fillna(global_mean)

train_clen = train_clean.drop(columns = [col])
test_clean = test_clean.drop(columns = [col])

/tmp/ipykernel_55/1588952097.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_clean[encoded_col_name] = train_clean[col].map(smoothed_encoding).fillna(global_mean)
/tmp/ipykernel_55/1588952097.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_clean[encoded_col_name] = train_clean[col].map(smoothed_encoding).fillna(global_mean)


In [128]:
list(test_clean.columns)

['SK_ID_CURR',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'DAYS_LAST_PHONE_CHANGE',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11'

In [129]:
smoothed_encoding

{'Advertising': np.float64(0.08156557675300376),
 'Agriculture': np.float64(0.10462958124779571),
 'Bank': np.float64(0.05196952252465978),
 'Business Entity Type 1': np.float64(0.08138259729639118),
 'Business Entity Type 2': np.float64(0.08527949334417956),
 'Business Entity Type 3': np.float64(0.09299443087254153),
 'Cleaning': np.float64(0.11039736368358759),
 'Construction': np.float64(0.11674450872003694),
 'Culture': np.float64(0.05605986682408393),
 'Electricity': np.float64(0.06646592520267568),
 'Emergency': np.float64(0.07159173367468184),
 'Government': np.float64(0.06979136625644024),
 'Hotel': np.float64(0.06435172970754985),
 'Housing': np.float64(0.07944989494426169),
 'Industry: type 1': np.float64(0.11039779618166698),
 'Industry: type 10': np.float64(0.06560746381990462),
 'Industry: type 11': np.float64(0.08651705534066642),
 'Industry: type 12': np.float64(0.0390693619909463),
 'Industry: type 13': np.float64(0.12736737915024218),
 'Industry: type 2': np.float64(0.

In [130]:
categorical_cols_updated[:-1]

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'WEEKDAY_APPR_PROCESS_START']

In [131]:
train_clean['NAME_CONTRACT_TYPE'].unique()

array(['Cash loans', 'Revolving loans'], dtype=object)

In [132]:
from sklearn.preprocessing import LabelEncoder

label_enoders = {}
label_mappings = {}

for col in categorical_cols_updated[:-1]:
    le = LabelEncoder()
    train_clean[col] = le.fit_transform(train_clean[col].astype(str))
    label_enoders[col] = le
    label_mappings[col] = dict(zip(le.classes_,le.transform(le.classes_)))    

In [133]:
from sklearn.preprocessing import LabelEncoder

label_enoders = {}
label_mappings = {}

for col in categorical_cols_updated[:-1]:
    le = LabelEncoder()
    test_clean[col] = le.fit_transform(test_clean[col].astype(str))
    label_enoders[col] = le
    label_mappings[col] = dict(zip(le.classes_,le.transform(le.classes_)))    

In [134]:
train_clean['NAME_CONTRACT_TYPE'].value_counts()

NAME_CONTRACT_TYPE
0    278232
1     29279
Name: count, dtype: int64

In [135]:
for col,mapping in label_mappings.items():
    print(f"\n{col} mapping:")
    print(mapping)


NAME_CONTRACT_TYPE mapping:
{'Cash loans': np.int64(0), 'Revolving loans': np.int64(1)}

CODE_GENDER mapping:
{'F': np.int64(0), 'M': np.int64(1)}

FLAG_OWN_CAR mapping:
{'N': np.int64(0), 'Y': np.int64(1)}

FLAG_OWN_REALTY mapping:
{'N': np.int64(0), 'Y': np.int64(1)}

NAME_INCOME_TYPE mapping:
{'Businessman': np.int64(0), 'Commercial associate': np.int64(1), 'Pensioner': np.int64(2), 'State servant': np.int64(3), 'Student': np.int64(4), 'Unemployed': np.int64(5), 'Working': np.int64(6)}

NAME_EDUCATION_TYPE mapping:
{'Academic degree': np.int64(0), 'Higher education': np.int64(1), 'Incomplete higher': np.int64(2), 'Lower secondary': np.int64(3), 'Secondary / secondary special': np.int64(4)}

NAME_FAMILY_STATUS mapping:
{'Civil marriage': np.int64(0), 'Married': np.int64(1), 'Separated': np.int64(2), 'Single / not married': np.int64(3), 'Widow': np.int64(4)}

NAME_HOUSING_TYPE mapping:
{'Co-op apartment': np.int64(0), 'House / apartment': np.int64(1), 'Municipal apartment': np.int64(

In [136]:
from sklearn.preprocessing import RobustScaler

In [137]:
scale_col = [c for c in train_clean.columns if c not in [TARGET,ID_COL] and train_clean[c].dtype in ['int64','float64']]
scaler = RobustScaler()

train_clean[scale_col[:-1]] = scaler.fit_transform(train_clean[scale_col[:-1]])
test_clean[scale_col[:-1]] = scaler.fit_transform(test_clean[scale_col[:-1]])

In [138]:
train_clean

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,age_group_middle,age_group_senior,employment_years,registration_years,ext_source_mean,ext_source_max,ext_source_min,ext_source_std,total_documents_provided,ORGANIZATION_TYPE_encoded
0,100002,1,0.0,1.0,0.0,0.0,0.0,0.615,-0.198521,-0.011205,...,-1.0,0.0,-0.233104,-0.156504,0.0,0.0,0.0,0.000000,0.0,0.092994
1,100003,0,0.0,0.0,0.0,-1.0,0.0,1.365,1.448012,0.597361,...,0.0,0.0,-0.010117,-0.606637,0.5,1.0,0.0,0.707107,0.0,0.059172
2,100004,0,1.0,1.0,1.0,0.0,0.0,-0.885,-0.702740,-1.004482,...,0.0,0.0,-0.399838,-0.044611,0.0,0.0,0.0,0.000000,-1.0,0.069791
3,100006,0,0.0,0.0,0.0,0.0,0.0,-0.135,-0.372874,0.264691,...,0.0,0.0,0.738972,0.974312,0.5,1.0,0.0,0.707107,0.0,0.092994
4,100007,0,0.0,1.0,0.0,0.0,0.0,-0.285,-0.000986,-0.168078,...,0.0,0.0,0.738567,-0.035287,0.5,1.0,0.0,0.707107,0.0,0.061129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0.0,1.0,0.0,-1.0,0.0,0.115,-0.480518,0.146912,...,-1.0,0.0,-0.395386,0.722552,0.5,1.0,0.0,0.707107,0.0,0.066124
307507,456252,0,0.0,0.0,0.0,0.0,0.0,-0.835,-0.452949,-0.713894,...,-1.0,1.0,-148.302711,-0.021209,0.5,1.0,0.0,0.707107,0.0,0.054001
307508,456253,0,0.0,0.0,0.0,0.0,0.0,0.065,0.304712,0.280876,...,0.0,0.0,2.714690,0.408264,0.0,0.0,0.0,0.000000,0.0,0.059172
307509,456254,1,0.0,0.0,0.0,0.0,0.0,0.265,-0.266266,-0.259960,...,-1.0,0.0,1.445973,-0.355060,0.0,0.0,0.0,0.000000,0.0,0.081383


In [139]:
numeric_cols[:100]

['CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'CNT_FAM_MEMBERS',
 'HOUR_APPR_PROCESS_START',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD

In [140]:
from scipy.stats import chi2_contingency, mannwhitneyu, pointbiserialr
from sklearn.feature_selection import mutual_info_classif

In [141]:
X_Stat =  train_clean.drop(columns=[ID_COL])
numerical_features = [c for c in X_Stat.columns if c!= TARGET and X_Stat[c].dtype in ['int64','float64']]
numerical_features

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'DAYS_LAST_PHONE_CHANGE',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUME

In [142]:
#PBC
corrs = []
for col in numerical_features[:50]:
    try:
        corr,pval = pointbiserialr(X_Stat[TARGET],X_Stat[col])
        corrs.append(
            {
                'feature': col,
                'correlation': abs(corr),
                'p_value': pval,
                'sig': 'Yes' if pval < 0.01 else 'No'
            }
        )
    except:
        continue

corr_df = pd.DataFrame(corrs).sort_values('correlation',ascending=False)

In [143]:
corr_df.head(20)

,feature,correlation,p_value,sig
13,DAYS_BIRTH,0.078239,0.000000e+00,Yes
25,REGION_RATING_CLIENT_W_CITY,0.060893,2.078455e-250,Yes
24,REGION_RATING_CLIENT,0.058899,2.172433e-234,Yes
34,DAYS_LAST_PHONE_CHANGE,0.055218,3.218029e-206,Yes
9,NAME_EDUCATION_TYPE,0.054699,2.159263e-202,Yes
1,CODE_GENDER,0.054692,2.403275e-202,Yes
16,DAYS_ID_PUBLISH,0.051457,2.527524e-179,Yes
32,REG_CITY_NOT_WORK_CITY,0.050994,3.805463e-176,Yes
8,NAME_INCOME_TYPE,0.046829,7.750686e-149,Yes
18,FLAG_EMP_PHONE,0.045982,1.442660e-143,Yes


In [144]:
X_Stat

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_INCOME_TYPE,...,age_group_middle,age_group_senior,employment_years,registration_years,ext_source_mean,ext_source_max,ext_source_min,ext_source_std,total_documents_provided,ORGANIZATION_TYPE_encoded
0,1,0.0,1.0,0.0,0.0,0.0,0.615,-0.198521,-0.011205,0.00,...,-1.0,0.0,-0.233104,-0.156504,0.0,0.0,0.0,0.000000,0.0,0.092994
1,0,0.0,0.0,0.0,-1.0,0.0,1.365,1.448012,0.597361,-0.75,...,0.0,0.0,-0.010117,-0.606637,0.5,1.0,0.0,0.707107,0.0,0.059172
2,0,1.0,1.0,1.0,0.0,0.0,-0.885,-0.702740,-1.004482,0.00,...,0.0,0.0,-0.399838,-0.044611,0.0,0.0,0.0,0.000000,-1.0,0.069791
3,0,0.0,0.0,0.0,0.0,0.0,-0.135,-0.372874,0.264691,0.00,...,0.0,0.0,0.738972,0.974312,0.5,1.0,0.0,0.707107,0.0,0.092994
4,0,0.0,1.0,0.0,0.0,0.0,-0.285,-0.000986,-0.168078,0.00,...,0.0,0.0,0.738567,-0.035287,0.5,1.0,0.0,0.707107,0.0,0.061129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,0.0,1.0,0.0,-1.0,0.0,0.115,-0.480518,0.146912,0.00,...,-1.0,0.0,-0.395386,0.722552,0.5,1.0,0.0,0.707107,0.0,0.066124
307507,0,0.0,0.0,0.0,0.0,0.0,-0.835,-0.452949,-0.713894,-1.00,...,-1.0,1.0,-148.302711,-0.021209,0.5,1.0,0.0,0.707107,0.0,0.054001
307508,0,0.0,0.0,0.0,0.0,0.0,0.065,0.304712,0.280876,0.00,...,0.0,0.0,2.714690,0.408264,0.0,0.0,0.0,0.000000,0.0,0.059172
307509,1,0.0,0.0,0.0,0.0,0.0,0.265,-0.266266,-0.259960,-1.50,...,-1.0,0.0,1.445973,-0.355060,0.0,0.0,0.0,0.000000,0.0,0.081383


In [64]:
mi_scores = mutual_info_classif(X_Stat[numerical_features],X_Stat[TARGET],random_state=42)
mi_df = pd.DataFrame(
    {
        'feature': numerical_features,
        'mi_score': mi_scores
    }
).sort_values('mi_score',ascending=False)

KeyboardInterrupt: 

In [145]:
train_clean.select_dtypes(include=['object']).columns.tolist()

['ORGANIZATION_TYPE']

In [146]:
train_clean.drop(columns=['ORGANIZATION_TYPE'],inplace=True)

In [147]:
train_clean.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,age_group_middle,age_group_senior,employment_years,registration_years,ext_source_mean,ext_source_max,ext_source_min,ext_source_std,total_documents_provided,ORGANIZATION_TYPE_encoded
0,100002,1,0.0,1.0,0.0,0.0,0.0,0.615,-0.198521,-0.011205,...,-1.0,0.0,-0.233104,-0.156504,0.0,0.0,0.0,0.000000,0.0,0.092994
1,100003,0,0.0,0.0,0.0,-1.0,0.0,1.365,1.448012,0.597361,...,0.0,0.0,-0.010117,-0.606637,0.5,1.0,0.0,0.707107,0.0,0.059172
2,100004,0,1.0,1.0,1.0,0.0,0.0,-0.885,-0.702740,-1.004482,...,0.0,0.0,-0.399838,-0.044611,0.0,0.0,0.0,0.000000,-1.0,0.069791
3,100006,0,0.0,0.0,0.0,0.0,0.0,-0.135,-0.372874,0.264691,...,0.0,0.0,0.738972,0.974312,0.5,1.0,0.0,0.707107,0.0,0.092994
4,100007,0,0.0,1.0,0.0,0.0,0.0,-0.285,-0.000986,-0.168078,...,0.0,0.0,0.738567,-0.035287,0.5,1.0,0.0,0.707107,0.0,0.061129


In [148]:
train_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 154 entries, SK_ID_CURR to ORGANIZATION_TYPE_encoded
dtypes: float64(152), int64(2)
memory usage: 361.3 MB


In [159]:
import re
train_clean = train_clean.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test_clean = test_clean.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [160]:
X = train_clean.drop(columns = ['TARGET'])
y = train_clean['TARGET']

In [161]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
    X,y,test_size=0.2,random_state=42
)

In [162]:
train_clean.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,age_group_middle,age_group_senior,employment_years,registration_years,ext_source_mean,ext_source_max,ext_source_min,ext_source_std,total_documents_provided,ORGANIZATION_TYPE_encoded
0,100002,1,0.0,1.0,0.0,0.0,0.0,0.615,-0.198521,-0.011205,...,-1.0,0.0,-0.233104,-0.156504,0.0,0.0,0.0,0.000000,0.0,0.092994
1,100003,0,0.0,0.0,0.0,-1.0,0.0,1.365,1.448012,0.597361,...,0.0,0.0,-0.010117,-0.606637,0.5,1.0,0.0,0.707107,0.0,0.059172
2,100004,0,1.0,1.0,1.0,0.0,0.0,-0.885,-0.702740,-1.004482,...,0.0,0.0,-0.399838,-0.044611,0.0,0.0,0.0,0.000000,-1.0,0.069791
3,100006,0,0.0,0.0,0.0,0.0,0.0,-0.135,-0.372874,0.264691,...,0.0,0.0,0.738972,0.974312,0.5,1.0,0.0,0.707107,0.0,0.092994
4,100007,0,0.0,1.0,0.0,0.0,0.0,-0.285,-0.000986,-0.168078,...,0.0,0.0,0.738567,-0.035287,0.5,1.0,0.0,0.707107,0.0,0.061129


In [163]:
test_clean.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_INCOME_TYPE,...,age_group_middle,age_group_senior,employment_years,registration_years,ext_source_mean,ext_source_max,ext_source_min,ext_source_std,total_documents_provided,ORGANIZATION_TYPE_encoded
0,100001,0.0,0.0,0.0,0.0,0.0,-0.20,0.286707,-0.290614,0.0,...,0.0,0.0,0.396327,0.122341,0.0,0.0,0.0,0.000000,0.0,0.092994
1,100005,0.0,1.0,0.0,0.0,0.0,-0.52,-0.548393,-0.454925,0.0,...,0.0,0.0,1.214996,0.832636,0.5,1.0,0.0,0.707107,0.0,0.059172
2,100013,0.0,1.0,1.0,0.0,0.0,0.40,0.514683,2.244032,0.0,...,0.0,0.0,1.210788,-0.416498,0.0,0.0,0.0,0.000000,0.0,0.069791
3,100028,0.0,0.0,0.0,0.0,2.0,1.40,2.715030,1.174971,0.0,...,0.0,0.0,0.219204,-0.447983,0.5,1.0,0.0,0.707107,0.0,0.092994
4,100038,0.0,1.0,1.0,-1.0,1.0,0.20,0.423545,0.301970,0.0,...,0.0,0.0,0.343535,-0.088157,0.5,1.0,0.0,0.707107,0.0,0.061129


In [152]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [164]:
from lightgbm import LGBMClassifier
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    classification_report,
    confusion_matrix
)

In [165]:
lgbm = LGBMClassifier (
    objective = "binary",
    boosting_type = "gbdt",
    n_estimators = 500,
    learning_rate = 0.05,
    num_leaves = 31,
    max_depth = 1,
    subsample = 0.8,
    colsample_bytree = 0.8,
    class_weight = "balanced",
    random_state = 42
)

lgbm.fit(
    X_train,
    y_train,
    eval_set=[(X_test,y_test)],
    eval_metric="auc"
    
)

[LightGBM] [Info] Number of positive: 19876, number of negative: 226132
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.149756 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4003
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 136
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

LGBMClassifier(class_weight='balanced', colsample_bytree=0.8,
               learning_rate=0.05, max_depth=1, n_estimators=500,
               objective='binary', random_state=42, subsample=0.8)

In [166]:
y_pred = lgbm.predict(X_test)
y_prob = lgbm.predict_proba(X_test)[:,1]

In [167]:
roc_auc_score(y_test,y_prob)

np.float64(0.6844051840353457)

In [168]:
accuracy_score(y_test,y_pred)

0.633920296570899

In [169]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.63      0.76     56554
           1       0.13      0.64      0.22      4949

    accuracy                           0.63     61503
   macro avg       0.54      0.63      0.49     61503
weighted avg       0.89      0.63      0.72     61503



In [170]:
import joblib
joblib.dump(
    lgbm,"lgbm_model.pkl"
)

['lgbm_model.pkl']

In [171]:
##load script
lgbm_loaded = joblib.load("lgbm_model.pkl")

In [173]:
y_prob = lgbm_loaded.predict_proba(X_test)[:,1]
y_prob

array([0.50844302, 0.40591457, 0.52587952, ..., 0.35026835, 0.46978387,
       0.43919615])